In [ ]:
#Start to use CASA

import os
os.chdir('analysis')

import os
import numpy as np

import shutil
import os
import math
import numpy as np
from pathlib import Path
import sys
import shutil


from casatasks import listobs
from casatasks import imhead
from casatasks import imstat
from casatasks import exportfits
from casatasks import imval
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np

import analysisUtils as au

#libraries
from astroquery.alma import Alma
alma = Alma()
from astropy import units as u
from astropy import coordinates
import numpy as np
from astroquery.nvas import Nvas
import requests
import shutil
import os

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel

import wget

import warnings
from astropy.utils.exceptions import AstropyWarning

from collections import Counter

warnings.simplefilter('ignore', category=AstropyWarning)

#phone text Function
import requests
def textme(words,choice):

    if choice=='go':
        bot_token = "enter bot token"
        chat_id = "enter chat id"

    url=f'https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={words}'

    requests.get(url)

#function: getimview image
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from casatasks import exportfits
import os

def getimview(imviewimage):
    output_file = 'test.fits'
    if os.path.exists(output_file):
        os.remove(output_file)

    exportfits(imagename=imviewimage, fitsimage=output_file, overwrite=True)

    hdul = fits.open(output_file)
    wcs = WCS(hdul[0].header, naxis=2)
    data = hdul[0].data
    data = data[0, 0, :, :] if data.ndim > 2 else data

    plt.figure(figsize=(8, 6))
    ax = plt.subplot(111, projection=wcs)
    im = ax.imshow(data, origin='lower', cmap='viridis', interpolation='none')
    plt.colorbar(im, ax=ax, label='Intensity')
    ax.set_title(f'{imviewimage}')
    ax.set_xlabel('Right Ascension')
    ax.set_ylabel('Declination')
    ax.coords.grid(True, color='white', ls='dotted')
    ax.coords[0].set_format_unit('hour')
    ax.coords[1].set_format_unit('deg')

    hdul.close()


In [ ]:
#Path on local computer to SMA Images
#Upper:3
#Lower: 4
#Middle: 2
parent_dir = Path.cwd().parent
smapaths=[f'{parent_dir}/Gaussian_Fit_Pipeline/220131_128/images/flag4_cal1/fields/science',
          f'{parent_dir}/Gaussian_Fit_Pipeline/220131_128/images/flag3_cal1/fields/science',
          f'{parent_dir}/Gaussian_Fit_Pipeline/230316_128/images/flag4_cal1/fields/science',
          f'{parent_dir}/Gaussian_Fit_Pipeline/230316_128/images/flag3_cal1/fields/science',]
          #f'{parent_dir}/Gaussian_Fit_Pipeline/220131_128/images/flag3_cal1/fields/science',]
#smapaths=[f'{parent_dir}/Gaussian_Fit_Pipeline/230316_128/images/flag2_cal1/fields/science]
#230316
#220131

In [ ]:
#function: get all files for a source
def allfiles(source):
    allVLApaths=os.listdir('VLA')
    allALMApaths=os.listdir('ALMA')

    finfiles=[]
    ifinfiles=[]

    iifinfiles=[]
    for iallVLA in allVLApaths:
        if source in iallVLA:
            ifinfiles=os.listdir(f'VLA/{iallVLA}')
            iifinfiles=[]
            for iiifinfiles in ifinfiles:
                if not 'fitfiles' in iiifinfiles:
                    iifinfiles.append(f'VLA/{iallVLA}/{iiifinfiles}')
    finfiles.append(iifinfiles)

    iifinfiles=[]  
    for iallALMA in allALMApaths:
        if source in iallALMA:
            ifinfiles=os.listdir(f'ALMA/{iallALMA}')
            iifinfiles=[]
            for iiifinfiles in ifinfiles:
                if not 'fitfiles' in iiifinfiles:
                    iifinfiles.append(f'ALMA/{iallALMA}/{iiifinfiles}')
    finfiles.append(iifinfiles)

    for smapath in smapaths:
        files = os.listdir(smapath)
        newpath3=[]
        for ifile in files:
            if source.lower() in ifile.lower():
                if 'image' in ifile:
                    newpath3=f'{smapath}/{ifile}'
        finfiles.append([newpath3])

    return(finfiles)


In [ ]:
#function: get multitude of coordinates
#since we are finding the pixel in a fit file from the same image we dont need to think about coordinate conversion
def get_multcoords(imfitloc_gauss2, tele,debugger):
    pixcoord1=[]
    pixcoord2=[]

    ia.close()
    ia.open(imfitloc_gauss2)
    stats=ia.statistics()

    # Extract the maximum pixel value
    max_val = stats['max'][0]

    # Get the pixel coordinates of the maximum
    max_pos = stats['maxpos']

    pix1 = stats['maxpos'][0]
    pix2 = stats['maxpos'][1]



    return(pix1,pix2,max_val)



'''
for imultgauss in multgauss:
    if source in imultgauss[0]:
        j=-1
        for iimultgauss in imultgauss:
            j=j+1
            if j==0:
                continue

            #offset_divisor=iimultgauss[0]
            iimultgauss=iimultgauss[0]
            icoords=[iimultgauss[1], iimultgauss[2]]
            icoord1=icoords[0].split(':')
            selectcoord1=f'{icoord1[0]}h{icoord1[1]}m{icoord1[2]}s'
            selectcoord2=icoords[1].replace(':','.')

            #aicoords=[]
            #aicoords=au.J2000ToICRS(f'{icoords[0]} {icoords[1]}',verbose=False).split(', ')
            #aicoord1=aicoords[0].split(':')
            #aselectcoord1=f'{aicoord1[0]}h{aicoord1[1]}m{aicoord1[2]}s'                
            #aselectcoord2=aicoords[1].replace(':','.')

            ia.close()
            ia.open(file)
            csys33 = ia.coordsys()

            #for VLA and SMA using original imagefile
            ia.close()
            ia.open(file)
            csys33 = ia.coordsys()
            pixcoords33=csys33.topixel([selectcoord1,selectcoord2])
            pixcoord1.append(pixcoords33['numeric'][0])
            pixcoord2.append(pixcoords33['numeric'][1])  
            
    
return(pixcoord1,pixcoord2)
'''

In [ ]:
#function: get highest resolution file
#allfile=[source,allfiles(source)]
#allfiles(source) is a 3D array, each element for a different telescope

#getbestrez(allfile,icoords,6)])
def sortfiles(getfiles,alltele,tele):
    i=-1
    j=-1
    gotit=0
    for telescopes in getfiles:
        i=i+1
        #iterate past the sourcename
        if i==0:
            continue
        k=-1
        for telescope in telescopes:
            k=k+1
            for file in telescope:
                if file==None:
                    continue
                if len(file)==0:
                    continue
                if tele!='NA':
                    bestrezfile=file
                    besttele=tele
                    return(bestrezfile,besttele)
                if 'fitfiles' in file:
                    if tele=='NA':
                        continue   
                breaker=0
                #mask files need to be filtered back... need to fix earlier in process.
                if 'mask' in file:
                    os.remove(file)
                    breaker=1
                if breaker==1:
                    continue
                j=j+1
                majunit=imhead(imagename=file)['restoringbeam']['major']['unit']
                majval=imhead(imagename=file)['restoringbeam']['major']['value']
                minunit=imhead(imagename=file)['restoringbeam']['minor']['unit']
                minval=imhead(imagename=file)['restoringbeam']['minor']['value']

                if majunit!='arcsec' or minunit!='arcsec':
                    if majunit=='deg':
                        majval=majval*60*60
                        majunit='arcsec'
                    if minunit=='deg':
                        minval=minval*60*60
                        minunit='arcsec'
                    else:
                        print(f'Beam Axis does not use arcsec or deg for units.  check file:\n{file}')
                
                #imhead appear to give diameter values (FWHM)
                beamrad=(majval)/2
                gotit=1
                if j==0:
                    bestrezfile=file
                    bestrez=beamrad
                    besttele=alltele[k]
                if j!=0:
                    if beamrad<bestrez:
                        bestrezfile=file
                        bestrez=beamrad  
                        besttele=alltele[k]
                i=i+1  
    if gotit==0:
        return(None,None)
    if gotit==1:
        return(bestrezfile,besttele)
    #send back coordinates based on the proper coordinate system


def getbestrez(getfiles,icoords,tele,alltele,choosefile,choicefile):
    while 0==0:
        if choosefile==0:
            bestrezfile,besttele=sortfiles(getfiles,alltele,tele)
        else:
            bestrezfile=choicefile
            besttele=tele
        if bestrezfile==None:
            return(None)
        if ':' in icoords[0]:
            icoord1=icoords[0].split(':')
            selectcoord1=f'{icoord1[0]}h{icoord1[1]}m{icoord1[2]}s' 
        else:
            selectcoord1=icoords[0]  
        if ':' in icoords[1]:
            selectcoord2=icoords[1].replace(':','.')
        else:
            selectcoord2=icoords[1]
        aicoords=[]
        aicoords=au.J2000ToICRS(f'{icoords[0]} {icoords[1]}',verbose=False).split(', ')
        aicoord1=aicoords[0].split(':')
        aselectcoord1=f'{aicoord1[0]}h{aicoord1[1]}m{aicoord1[2]}s'
        aselectcoord2=aicoords[1].replace(':','.')
        region=f'centerbox[[{selectcoord1},{selectcoord2}],3arcsec,3arcsec]]'
        aregion=f'centerbox[[{aselectcoord1},{aselectcoord2}],3arcsec,3arcsec]]'
        try:
            if besttele=='ALMA':
                if tele=='NA':
                    aicoords=[f"{imstat(bestrezfile,region=aregion)['maxposf'].split(',')[0]}", f"{imstat(bestrezfile,region=aregion)['maxposf'].split(',')[1].replace('.',':',2)}"]
                if tele!='NA':
                    aicoords=[f"{imstat(bestrezfile)['maxposf'].split(',')[0]}", f"{imstat(bestrezfile)['maxposf'].split(',')[1].replace('.',':',2)}"]
                aicoord1=aicoords[0].split(':')
                aselectcoord1=f'{aicoord1[0]}h{aicoord1[1]}m{aicoord1[2]}s'
                aicoord2=aicoords[1].split(':')
                aselectcoord2=f'{aicoord2[0]}d{aicoord2[1]}m{aicoord2[2]}s'

                icoords=au.ICRSToJ2000(f'{aicoords[0]} {aicoords[1]}',verbose=False).split(', ')
                icoord1=icoords[0].split(':')
                selectcoord1=f'{icoord1[0]}h{icoord1[1]}m{icoord1[2]}s'
                icoord2=icoords[1].split(':')
                selectcoord2=f'{icoord2[0]}d{icoord2[1]}m{icoord2[2]}s'
                break
            if besttele!='ALMA':
                if tele=='NA':
                    icoords=[f"{imstat(bestrezfile,region=region)['maxposf'].split(',')[0]}", f"{imstat(bestrezfile,region=region)['maxposf'].split(',')[1].replace('.',':',2)}"]
                if tele!='NA':
                    icoords=[f"{imstat(bestrezfile)['maxposf'].split(',')[0]}", f"{imstat(bestrezfile)['maxposf'].split(',')[1].replace('.',':',2)}"]
                icoord1=icoords[0].split(':')
                selectcoord1=f'{icoord1[0]}h{icoord1[1]}m{icoord1[2]}s'   
                icoord2=icoords[1].split(':')
                selectcoord2=f'{icoord2[0]}d{icoord2[1]}m{icoord2[2]}s'

                aicoords=au.J2000ToICRS(f'{icoords[0]} {icoords[1]}',verbose=False).split(', ')
                aicoord1=aicoords[0].split(':')
                aselectcoord1=f'{aicoord1[0]}h{aicoord1[1]}m{aicoord1[2]}s'
                aicoord2=aicoords[1].split(':')
                aselectcoord2=f'{aicoord2[0]}d{aicoord2[1]}m{aicoord2[2]}s'
                break
        except Exception as e:
            i=-1
            braker=-1
            if choosefile==0:
                for telescopes in getfiles:
                    i=i+1
                    #iterate past the sourcename
                    if i==0:
                        continue
                    k=-1
                    for telescope in telescopes:
                        k=k+1
                        j=-1
                        for file in telescope:
                            j=j+1
                            if file==bestrezfile:
                                getfiles[i][k][j]=None
                                braker=1
                                break
                        if braker==1:
                            break
                    if braker==1:
                        break


    sendback=[aselectcoord1,aselectcoord2],[selectcoord1,aselectcoord2],bestrezfile
    return(sendback)

    

In [ ]:
#function: make nicknames for the sources
def nickname(ivsource):
    if len(ivsource.split('MRK'))>1:
        ivsource=ivsource.split('MRK')[1]
    if len(ivsource.split('NGC'))>1:
        ivsource=ivsource.split('NGC')[1]
    if len(ivsource.split('J0'))>1:
        ivsource=ivsource.split('J0')[1]
    if len(ivsource.split('UGC'))>1:
        ivsource=ivsource.split('UGC')[1]
    if len(ivsource.split('ESO'))>1:
        ivsource=ivsource.split('ESO')[1]
    if len(ivsource.split('IC'))>1:
        ivsource=ivsource.split('IC')[1]
    if len(ivsource.split('B'))>1:
        ivsource=ivsource.split('B')[0]
    if len(ivsource.split('b'))>1:
        ivsource=ivsource.split('b')[0]
    if len(ivsource.split('-G009'))>1:
        ivsource=ivsource.split('-G009')[0]
    return(ivsource)

In [ ]:
#function: print the first line
def firstline():
    sta='target'
    stele='Telescope'
    sfl='flux(mJy)'
    ssma='snr'
    sfr='frequency(GHz)'
    sda='date (index)'
    sar='beam size (")'
    sfilename='file name'
    print(f"{sta.ljust(15)}{stele.ljust(15)}{sfr.ljust(19)}{sfl.ljust(16)}{sda.ljust(15)}{sar.ljust(19)}{ssma.ljust(15)}{sfilename}")


In [ ]:
#function: write image estimates
def write_region(iregion,pixcoord1,pixcoord2,apixcoord1,apixcoord2,tele,file,region,aregion,majval,minval,rotval,rotunit,write,ofixer,doublegaussfit,second_gauss_pix, debugger, fixtochosenfreq, alphas):
    worked1=0
    with open(iregion, 'w') as file2:
        if write==1:
            original_stdout2 = sys.stdout
            sys.stdout = file2

        if tele!='ALMA':
            try:
                if fixtochosenfreq==0 and alphas==0:
                    imstatter= imstat(imagename=file,region=region)
                    pixcoord1=imstatter['maxpos'][0]
                    pixcoord2=imstatter['maxpos'][1]
                else:
                    pixcoord1=pixcoord1[0]
                    pixcoord2=pixcoord2[0]
            except Exception as e:
                if write==1:
                    sys.stdout = original_stdout2
                return('bad',file)
            if ofixer==1:
                #fixer=',xyabp'
                fixer=',xyabp'
            if ofixer==0:
                fixer=',abp'
            if ofixer=='nofix':
                fixer=''

            if ofixer==0:
                try:
                    print(f'{imstat(imagename=file,region=region)["max"][0]},{pixcoord1},{pixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}') 
                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)

            if ofixer==1:
                try:
                    x = round(pixcoord1)
                    y = round(pixcoord2)
                    pix = f"{x},{y}"  # properly formatted box string
                    val = imval(imagename=file, box=pix)
                    val=val['data'][0]
                    print(f'{val},{pixcoord1},{pixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}') 

                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)

            if ofixer=='nofix':
                try:
                    print(f'{imstat(imagename=file,region=region)["max"][0]},{pixcoord1},{pixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}') 
                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)
            
        if tele=='ALMA':
            try:
                if fixtochosenfreq==0 and alphas==0:
                    imstatter= imstat(imagename=file,region=aregion)
                    apixcoord1=imstatter['maxpos'][0]
                    apixcoord2=imstatter['maxpos'][1]
                else:
                    apixcoord1=apixcoord1[0]
                    apixcoord2=apixcoord2[0]
            except Exception as e:
                if write==1:
                    sys.stdout = original_stdout2
                return('bad',file)
            if ofixer==1:
                #fixer=',xyabp'
                fixer=',xyabp'
            if ofixer==0:
                fixer=',abp'
            elif ofixer=='nofix':
                fixer=''

            if ofixer==0:
                try:
                    print(f'{imstat(imagename=file,region=aregion)["max"][0]},{apixcoord1},{apixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}')
                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)

            if ofixer==1:
                try:
                    x = round(apixcoord1)
                    y = round(apixcoord2)
                    pix = f"{x},{y}"  # properly formatted box string
                    val = imval(imagename=file, box=pix)
                    val=val['data'][0]
                    print(f'{val},{apixcoord1},{apixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}')
                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)

            if ofixer=='nofix':
                try:
                    print(f'{imstat(imagename=file,region=aregion)["max"][0]},{apixcoord1},{apixcoord2},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{fixer}') 
                except Exception as e:
                    if write==1:
                        sys.stdout = original_stdout2
                    return('bad',file)

        if doublegaussfit:
            doublefixer=''
            try:
                print(f'{second_gauss_pix[2]},{second_gauss_pix[0]},{second_gauss_pix[1]},{majval}arcsec,{minval}arcsec,{rotval}{rotunit}{doublefixer}') 
                worked1=1
            except Exception as e:
                if write==1:
                    sys.stdout = original_stdout2
                return('bad',file)
            if debugger:
                print('3.1.2')

        if write==1:
            sys.stdout = original_stdout2
    #worked1 shows that the file has no imstat under the proper region
    #worked2 shows that the regrid file has no imstat under the proper region
    return([worked1])


In [ ]:
#function: makegauss image
from casatools import componentlist
cl = componentlist()
from casatools import quanta
qa = quanta()

def makegauss(coords,freq,frequn,file,imfitter,tele,majorax,minorax,posang):

    ppas1=imfitter['pixelsperarcsec'][0]
    ppas2=imfitter['pixelsperarcsec'][1]

    aspp1=1/ppas1
    aspp2=1/ppas2

    #divide the grid into smaller pieces
    if aspp1<aspp2:
        aspp=aspp1
    else:
        aspp=aspp2

    fulpix1=imstat(file)['trc'][0]
    fulpix2=imstat(file)['trc'][1]

    if os.path.exists('testg.fits'):
        os.remove('testg.fits')
    if os.path.exists('testg.im'):
        shutil.rmtree('testg.im')
        os.mkdir('testg.im')

    ra=coords[0]
    dec=coords[1]
    #There doesn't appear to be a difference if the direction is J2000 or ICRS
    if tele=='ALMA':
        direction = f"ICRS {ra} {dec}"
    if tele!='ALMA':
        direction = f"J2000 {ra} {dec}"
    cl.done()

    cl.addcomponent(dir=direction, flux=1.0, fluxunit='Jy', freq=f'{freq}{frequn}', shape="Gaussian", majoraxis=majorax, minoraxis=minorax, positionangle=posang)

    aspp=f'{str(aspp)}arcsec'
    if os.path.exists("testg.im"):
        shutil.rmtree("testg.im")
    ia.fromshape("testg.im",[fulpix1,fulpix2],overwrite=True)
    cs=ia.coordsys()
    cs.setunits(['rad','rad'])
    cell_rad=qa.convert(qa.quantity(aspp),"rad")['value']

    cs.setincrement([-cell_rad,cell_rad],'direction')
    cs.setreferencevalue([qa.convert(ra,'rad')['value'],qa.convert(dec,'rad')['value']],type="direction")
    ia.setcoordsys(cs.torecord())
    ia.setbrightnessunit("Jy/pixel")
    ia.modify(cl.torecord(),subtract=False)
    exportfits(imagename='testg.im',fitsimage='testg.fits',overwrite=True)

In [ ]:
#function: makedot image
from casatools import componentlist
cl = componentlist()
from casatools import quanta
qa = quanta()

def makedot(coords,freq,frequn,file,imfitter,tele,type):

    ppas1=imfitter['pixelsperarcsec'][0]
    ppas2=imfitter['pixelsperarcsec'][1]

    aspp1=1/ppas1
    aspp2=1/ppas2

    #divide the grid into smaller pieces
    if aspp1<aspp2:
        aspp=aspp1
    else:
        aspp=aspp2

    fulpix1=imstat(file)['trc'][0]
    fulpix2=imstat(file)['trc'][1]

    if type=='orig':
        pointname='testp'
    if type=='fit':
        pointname='fitp'
    if os.path.exists(f'{pointname}.fits'):
        os.remove(f'{pointname}.fits')
    if os.path.exists(f'{pointname}.im'):
        shutil.rmtree(f'{pointname}.im')
        os.mkdir(f'{pointname}.im')
    ra=coords[0]
    dec=coords[1]
    #There doesn't appear to be a difference if the direction is J2000 or ICRS
    if tele=='ALMA':
        direction = f"ICRS {ra} {dec}"
    if tele!='ALMA':
        direction = f"J2000 {ra} {dec}"
    cl.done()

    cl.addcomponent(dir=direction, flux=1.0, fluxunit='Jy', freq=f'{freq}{frequn}', shape="Point")
    aspp=f'{str(aspp)}arcsec'
    if os.path.exists(f"{pointname}.im"):
        shutil.rmtree(f"{pointname}.im")
        os.mkdir(f"{pointname}.im")
    ia.fromshape(f"{pointname}.im",[fulpix1,fulpix2],overwrite=True)
    cs=ia.coordsys()
    cs.setunits(['rad','rad'])
    cell_rad=qa.convert(qa.quantity(aspp),"rad")['value']

    cs.setincrement([-cell_rad,cell_rad],'direction')
    cs.setreferencevalue([qa.convert(ra,'rad')['value'],qa.convert(dec,'rad')['value']],type="direction")
    ia.setcoordsys(cs.torecord())
    ia.setbrightnessunit("Jy/pixel")
    ia.modify(cl.torecord(),subtract=False)
    exportfits(imagename=f'{pointname}.im',fitsimage=f'{pointname}.fits',overwrite=True)

In [ ]:
#funcion: make dot with contour image
import os
from casatools import image
from casatasks import exportfits
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
from reproject import reproject_interp
from matplotlib.lines import Line2D

def makedotcontour(file, imfitloc, peak_value, beamsize, newfilename, printimages,color,pixlen,oicoords,freq2,frequnit,imfitter,alltele,tele, getchoosercoords, fixtochosenfreq, imagefound, prelimchecker, fit_sizenoloc, doublegaussfit,alphas):
    sepp=newfilename.split('.')
    source=sepp[0]
    freq=sepp[1]
    year=sepp[2]
    month=sepp[3]
    daynum=sepp[4].split('_')
    day=daynum[0]
    if isinstance(color, str):
        colors='one'
        bothcolorteller=False
    elif isinstance(color, list):
        bothcolorteller=True
        bothb=1
        bothr=1
        #first one is the selected fit... it may be red
        firstcolor=color[0]
        for icolor in color:
            if icolor=='r':
                bothb=0
            if icolor=='b':
                bothr=0
        if bothb==1:
            color='b'
        elif bothr==1:
            input('both are red')
            return('error')
        elif firstcolor=='1':
            color='b'
            if firstcolor=='b':
                input('the fixed location is a detection and the freely varying is a nondetection')
                return('error')
            
    # Define the paths
    wcs_main = None  # Initialize wcs_main to ensure it's accessible in the except block

    imagename = file
    contour_fits = "testp.fits"
    
    nicename=f'{source} {freq} {year}/{month}/{day}'

    # Zoom region size (in pixels, e.g., a square region of size `zoom_width x zoom_width`)
    zoom_width = pixlen

    # Temporary output FITS files
    output_file = "test.fits"
    imfit_fits = "imfit.fits"

    # Ensure previous files are cleared
    for f in [output_file, imfit_fits]:
        if os.path.exists(f):
            os.remove(f)

    try:
        # Export the main image and imfitloc as FITS files
        exportfits(imagename=imagename, fitsimage=output_file, overwrite=True)
        exportfits(imagename=imfitloc, fitsimage=imfit_fits, overwrite=True)          

        # Load main FITS file
        hdul_main = fits.open(output_file)
        wcs_main = WCS(hdul_main[0].header, naxis=2)
        data_main = hdul_main[0].data
        data_main = data_main[0, 0, :, :] if data_main.ndim > 2 else data_main
        data_main *= 1000  # Multiply intensity by 1000 to get into mJy
        img_shape = data_main.shape  # Shape of the main image (pixels)

        # Load imfitloc FITS file
        hdul_imfit = fits.open(imfit_fits)
        wcs_imfit = WCS(hdul_imfit[0].header, naxis=2)
        data_imfit = hdul_imfit[0].data
        data_imfit = data_imfit[0, 0, :, :] if data_imfit.ndim > 2 else data_imfit
    

        # Replace NaNs in data_imfit
        data_imfit = np.nan_to_num(data_imfit, nan=0.0)


        # Reproject imfit data onto main image WCS
        reprojected_data, _ = reproject_interp(
            (data_imfit, wcs_imfit),
            wcs_main,
            shape_out=img_shape
        )          

        #Load contour FITS file if it exists
        if os.path.exists(contour_fits):
            hdul_contour = fits.open(contour_fits)
            wcs_contour = WCS(hdul_contour[0].header, naxis=2)
            dot_ra = wcs_contour.wcs.crval[0]
            dot_dec = wcs_contour.wcs.crval[1]
            dot_pixel = wcs_main.world_to_pixel_values(dot_ra, dot_dec)
        else:
            dot_pixel = None

        dot_pixel2 = None 
        if color=='b' or fixtochosenfreq==1 or alphas==1:
            choosefile=1
            choicefile=imfitloc
            gausloc=getbestrez([source,[[imfitloc],[],[]]],oicoords,tele,alltele,choosefile,choicefile)
            #the gaussian location should be that of the imfitloc
            gauslocfinder=gausloc[2].replace('/fitfiles','')
            gauslocfinder=gauslocfinder.replace('.fits.fit','.fits')
            gauslocfinder_file=file.replace('.imfits','.fits')
            if gauslocfinder_file!=gauslocfinder:
                pass
                #textme(f"ERROR!",'go')
                #textme(f"{gauslocfinder}",'go')
                #textme(f"{gauslocfinder_file}",'go')
                #input('check')
            if gausloc=='None':
                return('None')
            if gausloc==None:
                return(None)
            type='fit'
            acoords=gausloc[0]
            coords=gausloc[1]
            if tele=='ALMA':
                makedot(acoords,freq2,frequnit,file,imfitter,tele,type)
            if tele!='ALMA':
                makedot(coords,freq2,frequnit,file,imfitter,tele,type)

            contour_fits2="fitp.fits" 
            if os.path.exists(contour_fits2):
                hdul_contour2 = fits.open(contour_fits2)
                wcs_contour2 = WCS(hdul_contour2[0].header, naxis=2)
                dot_ra2 = wcs_contour2.wcs.crval[0]
                dot_dec2 = wcs_contour2.wcs.crval[1]
                dot_pixel2 = wcs_main.world_to_pixel_values(dot_ra2, dot_dec2)
            else:
                dot_pixel2 = None
        
        # Plot the main image
        plt.figure(figsize=(8, 6))
        ax = plt.subplot(111, projection=wcs_main)
        im = ax.imshow(data_main, origin='lower', cmap='viridis', interpolation='none')
        # Adjust colorbar
        cbar = plt.colorbar(im, ax=ax, label='mJy/beam')

        # Define contour levels dynamically based on reprojected data
        vmin = np.nanmin(reprojected_data)
        vmax = np.nanmax(reprojected_data)

        # quick sanity: any finite pixels?
        finite_count = np.isfinite(reprojected_data).sum()

        if not np.isfinite(vmax) and not np.isfinite(vmin):
            print("No finite pixels at all. Skipping contour.")
            levels_imfit = []
        else:
            if vmax > 0:
                peak = vmax
            else:
                peak = vmin
            
            levels_imfit = [0.5 * peak]  # FWHM (always positive)

        #fwhm_level2 = 0.5 * np.max(reprojected_data)  # Full Width at Half Maximum
        #levels_imfit = [fwhm_level2]

        #fwhm_level2_1 = 0.01*np.nanmax(reprojected_data)  # Use nanmax to ignore NaNs
        #fwhm_level2_2 = 0.5*np.nanmax(reprojected_data)  # Use nanmax to ignore NaNs
        #fwhm_level2_3 = 0.9*np.nanmax(reprojected_data)  # Use nanmax to ignore NaNs
        #levels_imfit = [fwhm_level2_1,fwhm_level2_2,fwhm_level2_3] 

        # Extract and analyze the contours
        contours = ax.contour(reprojected_data, levels=levels_imfit, colors=color, origin="lower")

        #Define the image boundaries explicitly
        x_min, x_max = 0, img_shape[1] - 1
        y_min, y_max = 0, img_shape[0] - 1

        # Debug: Print boundary details

        # Threshold for boundary classification
        threshold = 90  # Percentage threshold


        # Check and classify vertices along the image edges
        for path in contours.get_paths():

            vertices = path.vertices  # Contour vertices (x, y)
            x_coords, y_coords = vertices[:, 0], vertices[:, 1]


            percentages=[]
            for x, y in zip(x_coords, y_coords):
                # Calculate proximity to the nearest boundary
                proximity_x = min(abs(x - x_min), abs(x - x_max)) / (x_max - x_min)
                proximity_y = min(abs(y - y_min), abs(y - y_max)) / (y_max - y_min)
                proximity = min(proximity_x, proximity_y)
                percentage = (1 - proximity) * 100
                percentages.append(percentage)
                # Classify as "on the boundary" if percentage >= threshold
            highest=0
            for percentage in percentages:
                if percentage>highest:
                    highest=percentage
            if highest > threshold:
                
                #print(f"x={x:.2f}, y={y:.2f} -> Proximity to boundary: {percentage:.2f}%")
                # Close FITS files
                hdul_main.close()
                hdul_imfit.close()
                if os.path.exists(contour_fits):
                    hdul_contour.close()
                if color=='b':
                    if os.path.exists(contour_fits2):
                        hdul_contour2.close()   
                return('ONBOUNDRY')
        # Create a custom blue line for the legend
        if bothcolorteller==False:
            if '*' not in peak_value:   
                blue_line = Line2D([0], [0], color=color, lw=1.5, label=f'0.5 contour of {peak_value} mJy Gaussian Peak')
            elif peak>=0:
                blue_line = Line2D([0], [0], color=color, lw=1.5, label=f'LOW SNR 0.5 Gaussian Contour: 3X RMS: {peak_value} mJy')
            elif peak<0:
                blue_line = Line2D([0], [0], color=color, lw=1.5, linestyle="--", label=f'LOW SNR 0.5 Gaussian Contour: 3X RMS: {peak_value} mJy')

        else:
            blue_line = Line2D([0], [0], color='b', lw=1.5, label=f'0.01, 0.5, 0.9 contour mJy 2-Component Gaussian Peak \n {peak_value[0]} (free) , {peak_value[1]} (fixed)')
        handles = [blue_line]
        # Overlay red dot if available
        if dot_pixel:
            ax.plot(dot_pixel[0], dot_pixel[1], 'ro', markersize=2, label="Reference Dot")
            if fixtochosenfreq==0 and alphas==0:
                locationofdot='Location from NED'
            elif imagefound==1:
                locationofdot='Location from chosen freq'
            elif imagefound==0:
                locationofdot='Location from NED (no chosen freq)'
            if doublegaussfit==1:
                locationofdot='Location from chosen freq'
            red_dot = Line2D([0], [0], color='red', marker='o', markersize=6, label=locationofdot, linestyle='')
            handles.append(red_dot)  
        if dot_pixel2:
            ax.plot(dot_pixel2[0], dot_pixel2[1], 'bo', markersize=2, label="Reference Dot2")
            blue_dot = Line2D([0], [0], color='blue', marker='o', markersize=6, label='Fitted Location for this image', linestyle='')
            handles.append(blue_dot)  

        black_line = Line2D([0], [0], color='black', lw=1.5, label=f'{beamsize}" FWHM beam')
        handles.append(black_line)
        # Add legend
        ax.legend(handles=handles, loc='upper right')
        # Apply zoom
        if dot_pixel:
            center_x, center_y = dot_pixel
            x_min = max(center_x - zoom_width / 2, 0)
            x_max = min(center_x + zoom_width / 2, img_shape[1])
            y_min = max(center_y - zoom_width / 2, 0)
            y_max = min(center_y + zoom_width / 2, img_shape[0])
            ax.set_xlim(x_min, x_max)
            ax.set_ylim(y_min, y_max)
        # Finalize plot
        if tele=='SMA':
            file='New Observation'
        ax.set_title(f'{file}\n{nicename}')
        ax.set_xlabel("Right Ascension")
        ax.set_ylabel("Declination")
        ax.coords.grid(True, color='white', ls='dotted')
        ax.coords[0].set_format_unit('hour')
        ax.coords[1].set_format_unit('deg')

        if prelimchecker:
            output_plot_filename=f'images/prelim/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/prelim/{source}'):
                os.makedirs(f'images/prelim/{source}')    
        elif fit_sizenoloc:
            output_plot_filename=f'images/noloc/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/noloc/{source}'):
                os.makedirs(f'images/noloc/{source}')               
        elif getchoosercoords:
            output_plot_filename=f'images/chosenfreq/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/chosenfreq/{source}'):
                os.makedirs(f'images/chosenfreq/{source}')
        elif fixtochosenfreq==1:
            output_plot_filename=f'images/fixedtochosen/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/fixedtochosen/{source}'):
                os.makedirs(f'images/fixedtochosen/{source}')
        elif doublegaussfit==1:
            output_plot_filename=f'images/doublegauss/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/doublegauss/{source}'):
                os.makedirs(f'images/doublegauss/{source}')
        elif alphas==1:
            output_plot_filename=f'images/alphas/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/alphas/{source}'):
                os.makedirs(f'images/alphas/{source}')

        breaker=0
        i=0
        while breaker==0:
            i=i+1
            newfilename=output_plot_filename.replace('.pdf',f'_{i}.pdf')
            newdate=f'{year}/{month}/{day}_{i}'
            if not os.path.exists(newfilename):
                breaker = 1

        output_plot_filename=newfilename

        plt.savefig(output_plot_filename, dpi=100, bbox_inches='tight')
        if printimages==True:
            plt.show()
        plt.close()
        # Close FITS files
        hdul_main.close()
        hdul_imfit.close()
        if os.path.exists(contour_fits):
            hdul_contour.close()
        if color=='b':
            if os.path.exists(contour_fits2):
                hdul_contour2.close()       

    except Exception as e:
        print(f"Error1: {e} for {file}")

    return(newdate)


In [ ]:
#function: make image with dot and no contour (not used by default)
import os
from casatasks import exportfits
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

def makeonlydotcontour(file, beamsize, fluxval, newfilename, printimages,pixlen):
    sepp=newfilename.split('.')
    source=sepp[0]
    freq=sepp[1]
    year=sepp[2]
    month=sepp[3]
    daynum=sepp[4].split('_')
    day=daynum[0]
    num=daynum[1]
    newdate=f'{year}/{month}/{day}'
    nicename=f'{source} {freq} {year}/{month}/{day} {num}'

    if printimages==True:

        # Define the paths
        contour_fits = "testp.fits"
        # Zoom region size (in pixels, e.g., a square region of size `zoom_width x zoom_width`)
        zoom_width = pixlen
        # Temporary output FITS file
        output_file = "test.fits"

        # Ensure previous files are cleared
        if os.path.exists(output_file):
            os.remove(output_file)

        try:
            # Export the main image as a FITS file
            exportfits(imagename=file, fitsimage=output_file, overwrite=True)

            # Load main FITS file
            with fits.open(output_file) as hdul_main:
                wcs_main = WCS(hdul_main[0].header, naxis=2)
                data_main = hdul_main[0].data
                data_main = data_main[0, 0, :, :] if data_main.ndim > 2 else data_main
                data_main *= 1000  # Scale intensity
                img_shape = data_main.shape  # Shape of the main image (pixels)

            # Load contour FITS file if it exists
            dot_pixel = None
            if os.path.exists(contour_fits):
                with fits.open(contour_fits) as hdul_contour:
                    wcs_contour = WCS(hdul_contour[0].header, naxis=2)
                    dot_ra = wcs_contour.wcs.crval[0]
                    dot_dec = wcs_contour.wcs.crval[1]
                    dot_pixel = wcs_main.world_to_pixel_values(dot_ra, dot_dec)

            # Plot the main image
            plt.figure(figsize=(8, 6))
            ax = plt.subplot(111, projection=wcs_main)
            im = ax.imshow(data_main, origin='lower', cmap='viridis', interpolation='none')
            plt.colorbar(im, ax=ax, label='mJy/beam')

            # Generate a legend for the beam size
            black_line = Line2D([0], [0], color='black', lw=1.5, label=f'{beamsize}" FWHM beam {fluxval} mJy upper bound')
            handles = [black_line]

            # Overlay red dot if available
            if dot_pixel:
                ax.plot(dot_pixel[0], dot_pixel[1], 'ro', markersize=3, label="Reference Dot")
                red_dot = Line2D([0], [0], color='red', marker='o', markersize=6, label='Fixed Location', linestyle='None')
                handles.append(red_dot)

            # Add legend
            ax.legend(handles=handles, loc='upper right')

            # Apply zoom using calculated region
            if dot_pixel:
                center_x, center_y = dot_pixel
                x_min = max(center_x - zoom_width / 2, 0)
                x_max = min(center_x + zoom_width / 2, img_shape[1])
                y_min = max(center_y - zoom_width / 2, 0)
                y_max = min(center_y + zoom_width / 2, img_shape[0])
                ax.set_xlim(x_min, x_max)
                ax.set_ylim(y_min, y_max)

            # Finalize plot
            if tele=='SMA':
                file='New Observation'
            ax.set_title(f'{file}\n{nicename}')
            ax.set_xlabel("Right Ascension")
            ax.set_ylabel("Declination")
            ax.coords.grid(True, color='white', ls='dotted')
            ax.coords[0].set_format_unit('hour')
            ax.coords[1].set_format_unit('deg')

            output_plot_filename=f'images/{source}/{newfilename}.pdf'
            if not os.path.exists(f'images/{source}'):
                os.makedirs(f'images/{source}')
            if os.path.exists(output_plot_filename):
                os.remove(output_plot_filename)
            plt.savefig(output_plot_filename, dpi=100, bbox_inches='tight')
            plt.show()

        except Exception as e:
            print(f"Plot error: {e} for {file}")
        
    return(newdate)

In [ ]:
#format beamsize number
def format_number(number):
        # Case 1: If the number is less than 1 (e.g., 0.004567)
        if number < 0:
                # Format the positive part of the number and prepend the negative sign
                return "-" + format_number(abs(number))
        if number < 1:
                # Convert the number to a string with high precision
                num_str = f"{number:.16g}"

                # Identify the leading zeros and decimal point
                leading_part = []
                for char in num_str:
                        if char == '0' or char == '.':
                                leading_part.append(char)
                        else:
                                break

                # Remove leading zeros and the decimal point for significant digits
                significant_digits = "".join(char for char in num_str if char.isdigit() and char != "0")

                # Extract the first two non-zero digits
                if len(significant_digits) >= 2:
                        first_two = significant_digits[:2]
                elif len(significant_digits) == 1:
                        first_two = significant_digits + "0"  # Pad with zero if only one significant digit exists
                else:
                        first_two = "00"  # Handle edge case like 0

                # Combine the leading zeros and the first two non-zero digits
                return "".join(leading_part) + first_two

        # Case 2: If the number is greater than or equal to 1
        else:
                # Format to two decimal places
                return f"{number:.2f}"


In [ ]:
#print statements and make picturesmakedotcontour
def printrueconv(imfitter,date,make,color,imfitloc,newfilename,printimages,beamsize,freq,oicoords,frequnit,telescopes, tele, file, source, getchoosercoords, fixtochosenfreq, imagefound, prelimchecker, fit_sizenoloc, doublegaussfit,writenoise, alphas):
    if doublegaussfit:
        low=[]
        detectregions=[]
    else:
        low=False
    nelem=imfitter['deconvolved']['nelements']
    counter=-1
    for inelem in range(nelem):
        if doublegaussfit:
            ilow=False
        counter=counter+1
        component=f'component{inelem}'
        fluxerr=imfitter['deconvolved'][component]['flux']['error'][0]
        fluxval=imfitter['deconvolved'][component]['flux']['value'][0]
        fluxval=fluxval*1000
        fluxun=imfitter['deconvolved'][component]['flux']['unit']

        #FWHM to sigma from wikipedia
        factorto3sig= 3 / (2 * np.sqrt(2 * np.log(2)))

        fitmajax=imfitter['results'][component]['shape']['majoraxis']['value']
        fitmajax=fitmajax*factorto3sig
        fitmajunit=imfitter['results'][component]['shape']['majoraxis']['unit']
        if fitmajunit=='deg':
            fitmajax=fitmajax*60*60
        

        fitminjax=imfitter['results'][component]['shape']['minoraxis']['value']
        fitminjax=fitminjax*factorto3sig
        fitminjunit=imfitter['results'][component]['shape']['minoraxis']['unit']
        if fitminjunit=='deg':
            noisefitminjax=fitminjax*60*60

        fitpos=imfitter['results'][component]['shape']['positionangle']['value']
        fitposun=imfitter['results'][component]['shape']['positionangle']['unit']
        if fitposun!='deg':
            input('pos not in deg')

        totstats = imstat(imagename=file)
        blc = totstats['blc']  # bottom-left corner: [x, y]
        trc = totstats['trc']  # top-right corner: [x, y]
        totregion = f'box[[{blc[0]}pix, {blc[1]}pix], [{trc[0]}pix, {trc[1]}pix]]'

        gausscen1=imfitter['results'][component]['pixelcoords'][0]
        gausscen2=imfitter['results'][component]['pixelcoords'][1]

        if not doublegaussfit:
            try:
                detectregion=f'ellipse[[{gausscen1}pix, {gausscen2}pix], [{fitmajax}arcsec, {fitminjax}arcsec], {fitpos}deg]'
                #noiseregion=f'{totregion}-{detectregion}'
                noiseregfile='noiseregfile.crtf'
                if os.path.exists(noiseregfile):
                    os.remove(noiseregfile)
                with open(noiseregfile, 'w') as noiseregfile:
                    if writenoise==True:
                        original_stdoutnoisereg = sys.stdout
                        sys.stdout = noiseregfile
                    print('#CRTFv0')
                    print(totregion)
                    print(f'-{detectregion}')
                    if writenoise==True:
                        sys.stdout = original_stdoutnoisereg
                #noisesnr=fluxval/imstat(imagename=file,region='noiseregfile.crtf')['rms'][0]
                noiseval=imstat(imagename=file,region='noiseregfile.crtf')['rms'][0]*1000
                if math.isnan(fluxerr):
                    tot_err=noiseval
                else:
                    fluxerr=fluxerr*1000
                    tot_err=np.sqrt(fluxerr**2 + noiseval**2)
                noisesnr=fluxval/tot_err

            except Exception as e:
                print(f'cannot make noise region for {file}:')
                print(e)

            if noisesnr<3:
                low=True

            fluxval=format_number(fluxval)
            sfluxval=fluxval
            sbeamsize=format_number(beamsize)

            if low!=True:
                if counter==0:
                    if make==True:
                        pixlen=100
                        date=makedotcontour(file,imfitloc,sfluxval,sbeamsize, newfilename, printimages,color,pixlen,oicoords,freq,frequnit,imfitter,telescopes,tele, getchoosercoords, fixtochosenfreq,imagefound, prelimchecker, fit_sizenoloc, doublegaussfit, alphas)
                        if date==None:
                            return(date,date)
                        if date=='ONBOUNDRY':
                            return(date,date)
                        stotarea=f'{beamsize:.5f}'
                        sfreq=f'{freq:.2f}'
                        ssnr=str(f'{noisesnr:.3f}')
                        if color=='r':
                            sfluxval=fluxval+'*'
                        print(f"{source.ljust(15)}{tele.ljust(15)}{sfreq.ljust(19)}{sfluxval.ljust(16)}{date.ljust(15)}{stotarea.ljust(19)}{ssnr.ljust(15)}{file}")
                        if make!=True:
                            date='NA'
                        return(low,date)
                    else:
                        input('multcoords function being used but shouldnt be')
            else:
                return(low,date)
        else:
            idetectregion='NA'
            try:
                idetectregion=f'ellipse[[{gausscen1}pix, {gausscen2}pix], [{fitmajax}arcsec, {fitminjax}arcsec], {fitpos}deg]'
            except Exception as e:
                print(f'cannot make noise region for {file}:')
                print(e)
            detectregions.append(idetectregion)
    




    if doublegaussfit:
        #the first gaussian is fixed to the chosen frequency
        #the second gaussian is free to vary, and centered on the location
        #chosen from the 'noloc' pipeline

        #the second gaussian SHOULD always converge (needs confirmation), and the first one
        #may or may not. if not, it should be considered a nondetection
        #the upperlimit should still be built into the pipeline

        #totdetectregion=f'{detectregions[0]}+{detectregions[1]}'
        #noiseregion=f'{totregion}-{totdetectregion}'

        totstats = imstat(imagename=file)
        blc = totstats['blc']  # bottom-left corner: [x, y]
        trc = totstats['trc']  # top-right corner: [x, y]
        totregion = f'box[[{blc[0]}pix, {blc[1]}pix], [{trc[0]}pix, {trc[1]}pix]]'
        noiseregfile='noiseregfile.crtf'
        if os.path.exists(noiseregfile):
            os.remove(noiseregfile)
        with open(noiseregfile, 'w') as noiseregfile:
            if writenoise==True:
                original_stdoutnoisereg = sys.stdout
                sys.stdout = noiseregfile
            print('#CRTFv0')
            print(totregion)
            print(f'-{detectregions[0]}')
            print(f'-{detectregions[1]}')
            if writenoise==True:
                sys.stdout = original_stdoutnoisereg
        
        noiseval=imstat(imagename=file,region='noiseregfile.crtf')['rms'][0]*1000

        colors=[]
        counter=-1
        sfluxarray=[]
        for inelem in range(nelem):
            counter=counter+1
            component=f'component{inelem}'
            fluxerr=imfitter['deconvolved'][component]['flux']['error'][0]
            fluxval=imfitter['deconvolved'][component]['flux']['value'][0]
            fluxun=imfitter['deconvolved'][component]['flux']['unit']
            tot_err=np.sqrt(fluxerr**2 + noiseval**2)
            noisesnr=fluxval/tot_err
            tot_err=np.sqrt(fluxerr**2 + noiseval**2)
            noisesnr=fluxval/tot_err
            if noisesnr<3:
                low=True
                print(f'bad double fit for {file}')
                return(low,date)
            if make==True:
                imajval = imfitter['results'][component]['shape']['majoraxis']['value']
                iminval = imfitter['results'][component]['shape']['minoraxis']['value']
                imajunit = imfitter['results'][component]['shape']['majoraxis']['unit']
                iminunit = imfitter['results'][component]['shape']['minoraxis']['unit']

                if imajunit!='arcsec':
                    if imajunit=='deg':
                        imajval=imajval*60*60
                        imajunit='arcsec'
                    else:
                        'check units'
                if iminunit!='arcsec':
                    if iminunit=='deg':
                        iminval=iminval*60*60
                        iminunit='arcsec'
                    else:
                        'check units'

                ibeamsize=np.sqrt(imajval*iminval)
                fluxval=fluxval*1000
                if fluxval<noiseval*3:
                    fluxval=noiseval*3
                    color='r'
                    colors.append('r')
                else:
                    color='b'
                    colors.append('b')
                sfluxval=format_number(fluxval)
                if color=='r':
                    sfluxval=sfluxval+'*'
                sfluxarray.append(sfluxval)
                sbeamsize=format_number(ibeamsize)
                
                stotarea=f'{ibeamsize:.5f}'

                sfreq=f'{freq:.2f}'
                
                ssnr=str(f'{noisesnr:.3f}')
                scounter=str(counter)
                print(f"{source.ljust(15)}{scounter.ljust(5)}{tele.ljust(15)}{sfreq.ljust(19)}{sfluxval.ljust(16)}{date.ljust(15)}{stotarea.ljust(19)}{ssnr.ljust(15)}{file}")

        if make==True:
            pixlen=100
            date=makedotcontour(file,imfitloc,sfluxarray,sbeamsize, newfilename, printimages,colors,pixlen,oicoords,freq,frequnit,imfitter,telescopes,tele, getchoosercoords, fixtochosenfreq,imagefound, prelimchecker, fit_sizenoloc, doublegaussfit, alphas)
            if date==None:
                return(date,date)
            if date=='ONBOUNDRY':
                return(date,date)
        if make!=True:
            date='NA'


        return(low,date)

In [ ]:
#function: check case of last letter
def ensure_last_char_uppercase(s):
    # Check if the string is not empty
    if s:
        # Get the last character
        last_char = s[-1]
        # Check if it is an alphabetical letter
        if last_char.isalpha():
            # Ensure it is uppercase
            s = s[:-1] + last_char.upper()
    return s


In [ ]:
#get FOV and beam
def compute_fov_and_beam(imagename):
      # Get header
      header = imhead(imagename)

      # Pixel dimensions
      nx = header['shape'][0]  # X (RA)
      ny = header['shape'][1]  # Y (Dec)

      # Pixel increment and units
      incr_x = abs(header['incr'][0])
      incr_y = abs(header['incr'][1])
      unit_x = header['axisunits'][0]
      unit_y = header['axisunits'][1]

      # Convert to arcsec
      def to_arcsec(value, unit):
            if unit == 'rad':
                  return value * 206265
            elif unit == 'deg':
                  return value * 3600
            elif unit == 'arcsec':
                  return value
            else:
                  raise ValueError(f"Unrecognized angular unit: '{unit}'")

      dx = to_arcsec(incr_x, unit_x)
      dy = to_arcsec(incr_y, unit_y)

      # Field of view
      fov_x = nx * dx
      fov_y = ny * dy
      fov_diag = (fov_x**2 + fov_y**2)**0.5

      # Beam parameters with unit check
      beam = header['restoringbeam']
      bmaj_val = beam['major']['value']
      bmin_val = beam['minor']['value']
      bmaj_unit = beam['major']['unit']
      bmin_unit = beam['minor']['unit']
      bpa = beam['positionangle']['value']  # degrees

      bmaj = to_arcsec(bmaj_val, bmaj_unit)
      bmin = to_arcsec(bmin_val, bmin_unit)
      bdiag = (bmaj**2 + bmin**2)**0.5  # beam diagonal in arcsec


      scalesize=fov_diag/bdiag

      return scalesize,fov_diag,bdiag



In [ ]:
#MAIN ENGINE

#all methods otherthan fixtochosenfreq uses this region for total region fitting (in arcseconds)
nofixmultval=10

#searches for sources that are throwing errors by looking for the max pixel in the proper region
#or throwing an error when no fitting region is set (big files are skipped)
#check bad/badimages to see if any should be added to totmasterbad
prelimchecker=0
#printbadfix prints images of the large files... it takes way to long
printbadfix=0

#this fits location to NED coordinates with a fixed beam size and location can vary
#this is used for when double gaussian fits are made as well as to find files outside of the NED coordinates 
#all error files are flagged, and can be seen at bad/badfiles/
fit_sizenoloc=0

#now we look for the best image sources. small beam detections
getchoosercoords=0
choosercoordsarray=[]
#nofixmultval must be adjust for certain sources
nofixmultval_exc=[['NGC3393', 1,]]
#nofixmultval_exc=[['NGC3393', 1,],['NGC2273',2.5]]
#nofixmultval_exc=[['x',0],['y',0]]

#this fits location to the chosen freq coordinates with a fixed beam size fixed location
fixtochosenfreq=0
#this method sets the fitting region to the size of the beam... fixmultval is the scale of the entire fitting region
#This value goes from FWHM to the 3 sigma of the beam
fixmultval=(3)/(2*np.sqrt(2*np.log(2)))

#too complicated for this data set
#set doublegaussfit to create a 2 componet gaussian fit.  The first component is
#from fixtochosenfreq, which will be fixed and the value of interest. This will appear
#as the first value (the first of 2 gaussians), the second gaussian is allowed to vary
#although the intial guess is that from the fit_sizenoloc fit file
###
#in the case that fixtochosenfreq is a nondetection and fit_sizenoloc is a detection, we will run the double gaussian fit
#there should be no case where fixtochosenfreq is a detection and fit_sizenoloc is a detection
#if both are non detections... we resort to the upper limit in fixtochosenfreq
doublegaussfit=0
#this sets the region total for the double guassian fitted image
dubmultval=3

alphas=1
fixmultval=(3)/(2*np.sqrt(2*np.log(2)))

###
###
###
###
debugger=0
#debugger=1

yestext=True
yestext=False

printimages=True
printimages=False

writesummary=True
writesummary=False

filechooser=False
#filechooser=True
filechoice='ALMA/NGC3393/member.uid___A001_X2fe_X675.ari_l.ngc3393_sci.spw0_1_2_3_235737MHz.12m.cont.I.pbcor.fits'

#this naming will be from the allsources array format
sourcechooser=False
#sourcechooser=True
sourcechoice='NGC3393'

telechooser=False
telechooser=True
telechoice='SMA'

printfile=False
#printfile=True

###
###
###
###

#text files are used for creating regions
#fit estimates
writerreg=1
#writerreg=0

#noise region for double gaussian fits
writerreglowdub=1
#writerreglowdub=0

#noise region for noise region 
writenoise=1
#writenoise=0

###
###
###
###
#fix the location of the gaussian. 0 for not fix 1 for fix.
###
###
###
###

if prelimchecker+fit_sizenoloc+getchoosercoords+fixtochosenfreq+doublegaussfit+alphas!=1:
    input('error in setup')

pipetype='NA'
#freely chosen fit
if prelimchecker==1:
    masterfixer='nofix'
    masterfixerlow='nofix'
    pipetype='prelim'

#fix the beam size (for all sources)
if fit_sizenoloc==1:
    masterfixer=0
    masterfixerlow=0
    pipetype='noloc'

#fix the beam size (chosen freq)
if getchoosercoords==1:
    masterfixer=0
    masterfixerlow=0
    pipetype='getchooser'

#fix the location and beam size
if fixtochosenfreq==1:
    masterfixer=0
    masterfixerlow=0
    pipetype='fixtochosen'

#fix the location and beam size
if alphas==1:
    masterfixer=0
    masterfixerlow=0
    pipetype='alphas'

#fix the location and beam size
if doublegaussfit==1:
    masterfixer=1
    masterfixerlow=1
    pipetype='doublegaussfit'
#the first gaussian is fixed to the chosen frequency
#the second gaussian is free to vary, and centered on the location
#chosen from the 'noloc' pipeline

#the second gaussian SHOULD always converge (needs confirmation), and the first one
#may or may not. if not, it should be considered a nondetection
#the upperlimit should still be built into the pipeline

###
###
###
###

from casatools import image
from casatasks import imfit
from casatasks import rmtables

import warnings
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.wcs import FITSFixedWarning
warnings.filterwarnings('ignore', category=FITSFixedWarning)

import os
from contextlib import contextmanager
import warnings
from astropy.wcs.wcs import FITSFixedWarning
from casatasks import casalog

warnings.filterwarnings('ignore', category=FITSFixedWarning)

@contextmanager
def silence_stderr_fd():
        saved_fd = os.dup(2)
        devnull = os.open(os.devnull, os.O_WRONLY)
        try:
                os.dup2(devnull, 2)
                yield
        finally:
                os.dup2(saved_fd, 2)
                os.close(saved_fd)
                os.close(devnull)


if prelimchecker:
    output_plot_filename=f'images/prelim'
elif fit_sizenoloc:
    output_plot_filename=f'images/noloc'          
elif getchoosercoords:
    output_plot_filename=f'images/chosenfreq'
elif fixtochosenfreq==1:
    output_plot_filename=f'images/fixedtochosen'
elif doublegaussfit==1:
    output_plot_filename=f'images/doublegauss'
elif alphas==1:
    output_plot_filename=f'images/alphas'

if os.path.exists(output_plot_filename):
    shutil.rmtree(output_plot_filename)

txtnamesandcoords='namesandcoords.txt'

ia = image()

source_text='namesandcoords.txt'

#[[sourcename,[[arcsec of tot fit region,ra,dec]]]]
#multgauss=[[[[]]]]
#multgauss=[['NGC3393',[[3.0,'10:48:23.409','-25:09:44.05']]]]
multgauss=[[[[]]]]

#The code will keep only names that come before a dot/decimal: "."
#specify strings to kill
listofkills=['Galaxy']

telescopes=['VLA','ALMA','SMA']
#telescopes=['ALMA']
indexfiles=[]

if os.path.exists('regrids'):
    shutil.rmtree('regrids')
os.makedirs('regrids')

if os.path.exists('guessregions'):
    shutil.rmtree('guessregions')
os.makedirs('guessregions')

#track file number
index=0
#holds the sources best resolution image
bestcoords=[]
badfiles=[]



#iterate through the 3 telescopes to get all sources
allsources=[]
oallsources=[]
for  tele in telescopes:
    if tele=='SMA':
        for smapath in smapaths:
            isources=os.listdir(smapath)
            sources=[]
            for iisources in isources:
                if 'image' in iisources:
                    iiisources=iisources.split('_test.image')[0]
                    oallsources.append(iiisources)
                    #ivsources=nickname(iiisources)
                    if len(iiisources.split('.'))>0:
                        iiisources=iiisources.split('.')[0]
                    for killstring in listofkills:
                        if killstring in iiisources:
                            iiisources=iiisources.replace(killstring,'')
                    iiisources=ensure_last_char_uppercase(iiisources)
                    sources.append(iiisources)
                    for i in sources:
                        allsources.append(i)

    if tele=='VLA':
        sources=os.listdir(tele)
        for i in sources:
            oallsources.append(i)
            if len(i.split('.'))>0:
                i=i.split('.')[0]
            for killstring in listofkills:
                if killstring in i:
                    i=i.replace(killstring,'')
            allsources.append(i)
            
    if tele=='ALMA':
        sources=os.listdir(f'{tele}')
        for i in sources:
            oallsources.append(i)
            if len(i.split('.'))>0:
                i=i.split('.')[0]
            for killstring in listofkills:
                if killstring in i:
                    i=i.replace(killstring,'')
            allsources.append(i)

allsources=np.unique(allsources)
oallsources=np.unique(oallsources)

if not os.path.exists('fitsumfiles/witherrors'):
    os.makedirs('fitsumfiles/witherrors')
summaryfile=f'fitsumfiles/witherrors/fitsummary_{pipetype}_witherrors.txt'

with open(summaryfile, 'w') as sumfile:
    if writesummary==True:
        original_stdoutsum = sys.stdout
        sys.stdout = sumfile

    #print the first line
    firstline()
    newsource=-1
    for source in allsources:
        breaker=0
        if sourcechooser==True:
            if source!=sourcechoice:
                breaker=1
        if breaker==1:
            continue
        newsource=newsource+1
        
        if not os.path.exists(source_text):
            input(f'{source_text} does not exist.  Go to the beginning cells and make it.')

        nonames=1
        with open(source_text, 'r') as infile:
            for params in infile:
                params=eval(params)
                for inamecoords in params:
                    oname=inamecoords[0][0]
                    nname=inamecoords[1][0][0]
                    ra=inamecoords[1][1]
                    dec=inamecoords[1][2]
                    if not source in oname:
                        nonames=0
                        continue
                    icoords=[]
                    icoords.append(ra)
                    icoords.append(dec)
                    oicoords=icoords.copy()
        if nonames==1:
            input(f'no names for {source}')
        allfile=[source,allfiles(source)]
        selectcoords=[]
        aselectcoords=[]


        #checking to see if bestcoords have already been obtained
        i=0
        for ibest in bestcoords:
            if ibest[0]==source:
                iibest=ibest
                i=i+1
        if i==0:
            namecoordtxt='namesandcoords.txt'
            if not os.path.exists(namecoordtxt):
                input(f'{namecoordtxt} Does Not Exist.  ')
            ibestcoords=[]
            if os.path.exists(namecoordtxt):
                if 0==0:
                    with open(txtnamesandcoords, 'r') as infile:
                        for params in infile:
                            params=eval(params)
                            for inamecoords in params:
                                oname=inamecoords[0]
                                nname=inamecoords[1][0]
                                ra=inamecoords[1][1]
                                dec=inamecoords[1][2]

                                icoord1=ra.split(':')
                                selectcoord1=f'{icoord1[0]}h{icoord1[1]}m{icoord1[2]}s'   
                                icoord2=dec.split(':')
                                selectcoord2=f'{icoord2[0]}d{icoord2[1]}m{icoord2[2]}s'

                                aicoords=au.J2000ToICRS(f'{ra} {dec}',verbose=False).split(', ')
                                aicoord1=aicoords[0].split(':')
                                aselectcoord1=f'{aicoord1[0]}h{aicoord1[1]}m{aicoord1[2]}s'
                                aicoord2=aicoords[1].split(':')
                                aselectcoord2=f'{aicoord2[0]}d{aicoord2[1]}m{aicoord2[2]}s'
                                sendback=[aselectcoord1,aselectcoord2],[selectcoord1,selectcoord2]
                                sendit=0
                                if source in oname:
                                    sendit=1
                                for checksource in oname:
                                    if source in checksource:
                                        sendit=1
                                if sendit==1:
                                    iibest=[]
                                    iibest.append(source)
                                    iibest.append(sendback)
                                    ibestcoords.append(iibest)

            if fixtochosenfreq==0 and alphas==0:
                choosefile=0
                choicefile='NA'
                imagefound=0
            if fixtochosenfreq==1 or alphas==1:
                choosefile=1
                folder = 'analysis/multi_freq_from_archive/images/chosenfreq'
                imagefound=0
                telefound=0
                for item in os.listdir(folder):
                    if 'pdf' in item:
                        continue
                    if source==item:
                        full_path = os.path.join(folder, item)
                        for item2 in os.listdir(full_path):
                            choicefile=os.path.join(full_path, item2)
                            imagefound=1
                            for findtele in telescopes:
                                if findtele in item2:
                                    chosentele=findtele
                                    telefound=1
            if imagefound==1:
                if telefound!=1:
                    print(telefound)
                    input('error in tele')
                #sendback=[aselectcoord1,aselectcoord2],[selectcoord1,aselectcoord2],bestrezfile
                #return(sendback)
                newval=[source,getbestrez(allfile,icoords,chosentele,telescopes,choosefile,choicefile)]
                if newval[1]!=None:
                    ibestcoords.append(newval)
                if newval[1]==None:
                    ibestcoords.append(f'{source} has no proper images')
            if imagefound==0:
                ibestcoords.append('no chosen frequency')
        bestcoords.append(ibestcoords)
        #verifying that bestcoords have been obtained
        i=0
        iibest=='NA'
        for ibest in bestcoords:
            if ibest[0][0]==source:
            # if the first entry is the source (the NED query location)
                iibest=ibest
                i=i+1

        #set icoords (the location of the Gaussian Fit) 
        # to the NED query if fixtochosenfreq==0 otherwise fix it to the location of the chosenfreq
        # if fixtochosenfreq==1 but there is no chosenfreq, the location from NED is chosen 
        if fixtochosenfreq==0 and alphas==0:
            icoords=iibest[0][1]  
            acoords=icoords[0]
            coords=icoords[1]
        elif imagefound==1:
            icoords=iibest[1]
            acoords=icoords[1][0]
            coords=icoords[1][1]
        elif imagefound==0:
            icoords=iibest[0][1]
            acoords=icoords[0]
            coords=icoords[1]


        #fill the files for the proper filename and telescope    
        if newsource!=0:
            if findbest==0:
                #input(f'cannotfind best file for {allsources[newsource-1]}')
                pass
            else:
                #input(f'best file for {allsources[newsource-1]} is {bestfile}')
                pass
        bestfile='NA'
        findbest=0
        for tele in telescopes:
            if telechooser==True:
                if tele!=telechoice:
                    continue
            files=[]
            if yestext==True:
                textme(f'{source}:{tele}','go')

            if tele=='SMA':
                files=[]
                smacount=-1
                for smapath in smapaths:
                    smacount=smacount+1
                    thisfiles = os.listdir(smapath)
                    i=0
                    newpath1=[]
                    for ifile in thisfiles:
                        if source.lower() in ifile.lower():
                            if 'image' in ifile:
                                i=i+1
                                newpath1=f'{smapath}/{ifile}'
                    if i==1:
                        files.append(newpath1)
                    if i==0:
                        files=[]
                    if i!=0:
                        if i!=1:
                            input(i)
                            input('error3')

            if tele=='ALMA':
                allALMApaths=os.listdir('ALMA')
                iifinfiles=[]
                for iallALMA in allALMApaths:
                    if source in iallALMA:
                        ifinfiles=os.listdir(f'ALMA/{iallALMA}')
                        iifinfiles=[]
                        for iiifinfiles in ifinfiles:
                            if not 'fitfiles' in iiifinfiles:
                                iifinfiles.append(f'ALMA/{iallALMA}/{iiifinfiles}')
                files=iifinfiles

            if tele=='VLA':
                allVLApaths=os.listdir('VLA')
                iifinfiles=[]
                for iallVLA in allVLApaths:
                    if source in iallVLA:
                        ifinfiles=os.listdir(f'VLA/{iallVLA}')
                        iifinfiles=[]
                        for iiifinfiles in ifinfiles:
                            if not 'fitfiles' in iiifinfiles:
                                iifinfiles.append(f'VLA/{iallVLA}/{iiifinfiles}')
                files=iifinfiles

            #iterating through all of the files
            for file in files:
                if alphas==1:
                    if 'flag3' in file:
                        sideband='UP'
                    elif 'flag4' in file:
                        sideband='DOWN'
                    elif 'flag2' in file:
                        sideband='both'
                    elif 'flag5' in file:
                        sideband='both'
                    else:
                      input('SMA error 4')

                if debugger:
                        print('1')
                #skip bad files flagged from prelimchecker
                breaker=0
                if prelimchecker==1:
                    scalesize=compute_fov_and_beam(file)
                    scalesizeval=scalesize[0]
                    if scalesizeval>=60:
                        badfiles.append(['bigsize',file])
                        continue
                else:
                    if fit_sizenoloc==1:
                        badfileversions=['prelim']
                    elif getchoosercoords==1:
                        badfileversions=['prelim','sizenoloc']
                    elif fixtochosenfreq==1:
                        badfileversions=['prelim','sizenoloc']
                    elif doublegaussfit==1:
                        badfileversions=['prelim','sizenoloc']
                    elif alphas==1:
                        badfileversions=['prelim','sizenoloc']
                    for ibad in badfileversions:
                        badfiletxt=f'bad/masterbadfiles/masterbadfiles_{ibad}.txt'
                        if os.path.exists(badfiletxt):
                            with open(badfiletxt, 'r') as lines:
                                for line in lines:
                                    line=line.replace('\n','')
                                    if line==file:
                                        if tele=='SMA':
                                            input('SMA error')
                                        breaker=1
                if breaker==1:
                    continue
                
                if printfile==True:
                    print(file)
                breaker=0
                if filechooser==True:
                    if file!=filechoice:
                        breaker=1
                if breaker==1:
                    continue
                if not os.path.exists(file):
                    input(f'{file} doesnt exist')
                #get frequency, date
                #print(imhead(imagename=file)['axisnames'])
                axisnames=imhead(imagename=file)['axisnames']
                i=-1
                for iaxisname in axisnames:
                    i=i+1
                    if iaxisname=='Frequency':
                        findfreq=i
                if i==-1:
                    input('error1')

                if not alphas==1:
                    freq=imhead(imagename=file)['refval'][findfreq]/10**9
                elif  sideband=='both':
                    freq=imhead(imagename=file)['refval'][findfreq]/10**9
                elif sideband=='UP':
                    freq=235.55
                elif sideband=='DOWN':
                    freq=215.55
                


                if getchoosercoords:
                    if not 200<=freq<=400:
                        continue
                if imhead(imagename=file)['axisunits'][findfreq]!='Hz':
                    input('error2')         



                #obtain beam properties for this image
                imheader=imhead(imagename=file)
                if debugger:
                    print('2')
                majunit=imheader['restoringbeam']['major']['unit']
                majval=imheader['restoringbeam']['major']['value']
                minunit=imheader['restoringbeam']['minor']['unit']
                minval=imheader['restoringbeam']['minor']['value']
                rotunit=imheader['restoringbeam']['positionangle']['unit']
                if rotunit!='deg':
                    print(f'Beam Rotation does not use deg for units.  check imhead for: {file}')
                rotval=imheader['restoringbeam']['positionangle']['value']
                if majunit!='arcsec':
                    if majunit=='deg':
                        majval=majval*60*60
                        majunit='arcsec'
                    else:
                        'check units'
                if minunit!='arcsec':
                    if minunit=='deg':
                        minval=minval*60*60
                        minunit='arcsec'
                    else:
                        'check units'
                #imhead appear to give diameter values (FWHM)
                beamsize=np.sqrt(majval*minval)
                if getchoosercoords:
                    if beamsize>=1:
                        continue

                #for a single gaussian fit
                #consider a region "multiplier" times the size of the beam for a compact source
                #minval and majval which are FWHM (diamaters)
                if fixtochosenfreq==1 or alphas==1:
                    multipliertot=fixmultval
                else:
                    multipliertot=nofixmultval

                offset1tot=majval*multipliertot
                offset2tot=minval*multipliertot

                if getchoosercoords==1 and offset1tot<1:
                    offset1tot=1
                if getchoosercoords==1 and offset2tot<1:
                    offset2tot=1

                if getchoosercoords==1:
                    #nofixmultval must be adjust for certain sources
                    for inofix in nofixmultval_exc:
                        if inofix[0].lower() in source.lower():
                            offset1tot=inofix[1]


                #if the image is a very high resolution than the coordinates from
                #NED may not be a good guess... in which case we will use the image from the highest obs
                #if getchoosercoords==1:
                    pass
                    #choosefile=1
                    #choicefile=file
                    #xacoords,xcoords,checkthisfile=getbestrez('NA',coords,tele,'NA',choosefile,choicefile)                
                #elif getchoosercoords==1:
                    #choosefile=1
                    #choicefile=file
                    #xacoords,xcoords,checkthisfile=getbestrez('NA',coords,tele,'NA',choosefile,choicefile)                
                #else:
                    #input(f'error for {source} in getchoosercoords')
                

                #this is the region of the beam                 
                regionsmall=f'rotbox[{coords},[{majval}arcsec,{minval}arcsec],{rotval}{rotunit}]'
                aregionsmall=f'rotbox[{acoords},[{majval}arcsec,{minval}arcsec],{rotval}{rotunit}]'

                namefreq=int(freq)
                frequnit='GHz'
                date=imhead(imagename=file, mode="get", hdkey="date-obs").split('/')

                index=index+1

                #create a new file for different coordinate systems and make a new filename
                newfilename=f'{source}.{str(namefreq)}GHz.{date[0]}.{date[1]}.{date[2]}'
                #mapping from old filename, new filename, and index
                ifile=[index,file,newfilename]
                indexfiles.append(ifile)

                #for ALMA
                if tele=='ALMA':
                    ia.close()
                    ia.open(file)
                    csys = ia.coordsys()
                    apixcoords=csys.topixel(acoords)
                    apixcoord1=[]
                    apixcoord2=[]
                    #input(pixcoords['numeric'][0])
                    apixcoord1.append(apixcoords['numeric'][0])
                    apixcoord2.append(apixcoords['numeric'][1])
                    pixcoord1='NA"'
                    pixcoord2='NA'

                #for others
                else:
                    ia.close()
                    ia.open(file)
                    csys = ia.coordsys()
                    #for VLA and SMA using original imagefile
                    ia.close()
                    ia.open(file)
                    csys = ia.coordsys()
                    pixcoords=csys.topixel(coords)
                    pixcoord1=[]
                    pixcoord2=[]
                    #input(pixcoords['numeric'][0])
                    pixcoord1.append(pixcoords['numeric'][0])
                    pixcoord2.append(pixcoords['numeric'][1])
                    apixcoord1='NA"'
                    apixcoord2='NA'

                if debugger:
                    print('3')
                filename=file.split('/')
                filename=filename[len(filename)-1]

                for oi in oallsources:
                    if source in oi:
                        if getchoosercoords:
                            fitfilename=f'images/chosenfreq/{source}'
                            imfitloc=f'images/chosenfreq/{source}/{filename}.fit'       
                        else:
                            fitfilename=f'{tele}/{oi}/fitfiles'
                            imfitloc=f'{fitfilename}/{filename}_{pipetype}.fit'
                        if doublegaussfit:
                            imfitloc_gauss2=f'{fitfilename}/{filename}_noloc.fit'
                            #output will be in same coordinate system as the current file... since it is a fit from the same telescope
                            if debugger:
                                print('3.0.0')
                            second_gauss_pix=get_multcoords(imfitloc_gauss2, tele, debugger)
                            if debugger:
                                print('3.0.1')
                        else:
                            imfitloc_gauss2='NA'
                            second_gauss_pix='NA'
                            
                if not os.path.exists(fitfilename):
                    os.makedirs(fitfilename)

                iregion=f'guessregions/{index}region.txt'
                if os.path.exists(iregion):
                    os.remove(iregion)
                Path(iregion).touch()


                #print('\nbeginning imstats\n')
                fixer=masterfixer

                regiontot=f'centerbox[{coords},[{offset1tot}arcsec,{offset1tot}arcsec]]'
                aregiontot=f'centerbox[{acoords},[{offset1tot}arcsec,{offset1tot}arcsec]]'

                worked=write_region(iregion,pixcoord1,pixcoord2,apixcoord1,apixcoord2,tele,file,regiontot,aregiontot,majval,minval,rotval,rotunit,writerreg,fixer,doublegaussfit,second_gauss_pix, debugger, fixtochosenfreq,alphas)
                if worked[0]=='bad':
                    badfiles.append([6,file])
                    continue
                if debugger:
                    print('4')
                if worked=='skip':
                    continue

                #for extended sources 10" should be used
                #otherwise a region twice the beam size should suffice (compact)
                #for a multiple gaussian fit
                if worked[0]==1:
                    offset1tot=10
                    offset2tot=10
                    regiontot=f'rotbox[{coords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'
                    aregiontot=f'rotbox[{acoords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'

                log_file = 'casa_log.txt' 
                casalog.setlogfile(log_file)
                if fit_sizenoloc:
                    try:
                        imstat(file,region=regiontot)
                    except Exception as e:
                        badfiles.append([0,file])
                        print(f"Error with region {file} ... likely not the location of interest")
                        continue
                breaker=0
                if tele=='ALMA':
                    with silence_stderr_fd():
                            try:
                                imfitter=imfit(imagename=file,model=imfitloc,region=aregiontot,estimates=iregion)
                            except Exception as e:
                                breaker=1
                    if breaker==1:
                        badfiles.append([1,file])
                        print(f'imfit does not work for {file} 1')
                if tele!='ALMA':       
                    with silence_stderr_fd():        
                          try:
                              imfitter=imfit(imagename=file,model=imfitloc,region=regiontot,estimates=iregion)
                          except Exception as e:
                              breaker=1
                    if breaker==1:
                        badfiles.append([1,file])
                        print(f'imfit does not work for {file} 1')
                if prelimchecker==1:
                    #the following doesn't hold much weight since prelimchecker just throws an error for bad files
                    pos_thresher=0
                    elongthresher=0
                    if breaker!=1:
                        if imfitter['converged'][0]==True:
                            fitaxmaj=imfitter['results']['component0']['shape']['majoraxis']['value']
                            fitaxmajun=imfitter['results']['component0']['shape']['majoraxis']['unit']
                            fitaxmin=imfitter['results']['component0']['shape']['minoraxis']['value']
                            fitaxminun=imfitter['results']['component0']['shape']['minoraxis']['unit']

                            if fitaxmajun=='deg':
                                fitaxmaj=fitaxmaj*60*60
                            if fitaxminun=='deg':
                                fitaxmin=fitaxmin*60*60

                            fitaxang=imfitter['results']['component0']['shape']['positionangle']['value']
                            fitaxangun=imfitter['results']['component0']['shape']['positionangle']['unit']
                            fitaxangerr=imfitter['results']['component0']['shape']['positionangleerror']['value']
                            fitaxangerrun=imfitter['results']['component0']['shape']['positionangleerror']['unit']



                if breaker==1:
                    continue

                conv_test=False
                if imfitter['converged'][0]==True:
                    conv_test=True
                
                if conv_test==False:
                    #if fixtochosenfreq!=1:
                    #    badfiles.append([4,file])
                    #    breaker=1
                    #else:
                        low=True
                if breaker==1:
                    continue

                    '''
                    low=True

                    if fixtochosenfreq==1:
                        multipliertot=1
                    else:
                        multipliertot=10
                    offset1tot=majval*multipliertot
                    offset2tot=minval*multipliertot

                    regiontot=f'rotbox[{coords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'
                    aregiontot=f'rotbox[{acoords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'


                    imfitter2='NA'
                    breaker=0
                    if tele=='ALMA':
                        try:
                            imfitter2=imfit(imagename=file,model=imfitloc,region=aregiontot,estimates=iregion)
                        except Exception as e:
                            badfiles.append([2,file])
                            print(f'imfit does not work for {file} 2\n')
                            breaker=1
                    if tele!='ALMA':                  
                        try:
                            imfitter2=imfit(imagename=file,model=imfitloc,region=regiontot,estimates=iregion)
                        except Exception as e:
                            badfiles.append([2,file])
                            print(f'imfit does not work for {file} 2\n')
                            breaker=1

                    if imfitter2['converged'][0]==True: 
                        imfitter=imfitter2
                        conv_test=True
                    
                if breaker==1:
                    continue
                '''
                    
                breaker2=0
                keyword = 'Found no pixels over which to sum for component 0'

                with open(log_file, 'r') as f:
                    log_content = f.read()
                if keyword in log_content:
                    breaker2=1
                os.remove(log_file)
                if breaker2==1:
                    #badfiles.append([8,file])
                    #continue
                    pass
                majorax=f'{majval}{majunit}'
                minorax=f'{minval}{minunit}'
                posnang=f'{rotval}{rotunit}'
                if tele=='ALMA':
                    #Here you can see the difference if you change coords to acoords
                    type='orig'
                    makedot(acoords,freq,frequnit,file,imfitter,tele,type)
                    makegauss(acoords,freq,frequnit,file,imfitter,tele,majorax,minorax,posnang)
                else:
                    type='orig'
                    makedot(coords,freq,frequnit,file,imfitter,tele,type)
                    makegauss(coords,freq,frequnit,file,imfitter,tele,majorax,minorax,posnang)

                #fit a function to the dot to simply get a contour, but color it red
                for oi in oallsources:
                    if source in oi:
                        imfitloc2=f'{tele}/{oi}/fitfiles/{filename}_g.fit'
                if tele=='ALMA':
                    with silence_stderr_fd():        
                        imfit(imagename='testg.fits',model=imfitloc2,region=aregiontot)
                if tele!='ALMA':
                    with silence_stderr_fd():  
                        imfit(imagename='testg.fits',model=imfitloc2,region=regiontot)

                if conv_test==True:
                    writeandstore=1
                    color='b'
                    low,date=printrueconv(imfitter,'NA',writeandstore,color,imfitloc,newfilename,printimages,beamsize,freq,oicoords,frequnit,telescopes, tele, file, source, getchoosercoords, fixtochosenfreq, imagefound, prelimchecker, fit_sizenoloc, doublegaussfit,writenoise, alphas)
                    if not low==True and getchoosercoords==1:
                        foundsource=0
                        if len(choosercoordsarray) != 0:
                          i=-1
                          for ichoose in choosercoordsarray:
                              i=i+1
                              if source==ichoose[0]:
                                  foundsource=1
  
                                  if beamsize<ichoose[1]:
                                      choosercoordsarray[i]=[source,beamsize,file,newfilename, tele]
                        if foundsource==0:
                            choosercoordsarray.append([source,beamsize,file,newfilename, tele])
                                             
                        
                        
                    if date=='ONBOUNDRY':
                        if fixtochosenfreq==1:
                            badfiles.append([5,file])
                            print(f'ON BOUNDRY: {file}')
                            continue
                        if alphas==1:
                            input('SMA error2')
                            continue
                    if worked[0]==1:
                        if low==True:
                            #The double convolution may not be needed and should not be used.  Try a single imfitter image
                            #consider a region "multiplier" times the size of the beam for a compact source
                            multipliertot=dubmultval
                            offset1tot=majval*multipliertot
                            offset2tot=minval*multipliertot
                            regiontot=f'rotbox[{coords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'
                            aregiontot=f'rotbox[{acoords},[{offset1tot}arcsec,{offset1tot}arcsec],{rotval}{rotunit}]'

                            iregion=f'guessregions/{index}region.txt'
                            if os.path.exists(iregion):
                                os.remove(iregion)
                            Path(iregion).touch()

                            worked=write_region(iregion,[pixcoord1[0]],[pixcoord2[0]],[apixcoord1[0]],[apixcoord2[0]],tele,file,regiontot,aregiontot,majval,minval,rotval,rotunit,writerreglowdub,fixer,doublegaussfit,second_gauss_pix, debugger, fixtochosenfreq,alphas)
                            if worked[0]=='bad':
                                badfiles.append([7,file])
                                continue
                            with silence_stderr_fd():  
                                try:
                                    imfitter=imfit(imagename=file,model=imfitloc,region=regiontot,estimates=iregion)
                                except Exception as e:
                                    breaker=1
                            if breaker==1:
                                badfiles.append([3,file])
                                print(f'imfit does not work for {file} 3')
                            if imfitter['converged'][0]==True:
                                conv_test=True
                            if conv_test==True:
                                writeandstore=1
                                color='r'
                                low2,date2=printrueconv(imfitter,'NA',writeandstore,color,imfitloc,newfilename,printimages,beamsize,freq,oicoords,frequnit,telescopes, tele, file, source, getchoosercoords, fixtochosenfreq,imagefound, prelimchecker, fit_sizenoloc, doublegaussfit,writenoise, alphas)
                                if date2=='ONBOUNDRY':
                                    print(f'{file} on boundry')
                                    continue
                                #low=True
                if low==True:
                    noise=imstat(imagename=file,region=regionsmall)['rms'][0]
                    upbound=noise*3
                    fluxval=upbound*1000
                    fluxval=format_number(fluxval)
                    sfluxval=f'{fluxval}*'
                    sbeamsize=format_number(beamsize)
                    color='red'
                    pixlen=100
                    date=makedotcontour(file,imfitloc,sfluxval,sbeamsize, newfilename, printimages,color,pixlen,oicoords,freq,frequnit,imfitter,telescopes,tele, getchoosercoords, fixtochosenfreq,imagefound, prelimchecker, fit_sizenoloc, doublegaussfit, alphas)

                    if date=='ONBOUNDRY':
                        print(f'{file} on boundry')
                        continue 
                    #date=makeonlydotcontour(file, sbeamsize, sfluxval, newfilename, printimages)
                    blank='NA'
                    stotarea=f'{beamsize:.5f}'
                    sfreq=f'{freq:.2f}'
                    print(f"{source.ljust(15)}{tele.ljust(15)}{sfreq.ljust(19)}{sfluxval.ljust(16)}{date.ljust(15)}{stotarea.ljust(19)}{blank.ljust(15)}{file}")

    if writesummary==True:
        sys.stdout = original_stdoutsum
            


In [ ]:
#adjust fitsummaries to skip error files
import os
import sys

os.chdir('analysis/multi_freq_from_archive')

textin=f'fitsumfiles/witherrors/fitsummary_{pipetype}_witherrors.txt'
if not os.path.exists('fitsumfiles'):
    os.makedirs('fitsumfiles/noerrors')
textout=f'fitsumfiles/noerrors/fitsummary_{pipetype}_noerrors.txt'

if os.path.exists(textout):
    os.remove(textout)

write=True
#write=False
with open(textin, 'r') as infile:
    with open(textout, 'w') as outfile:
        for line in infile:
            if 'Error' in line:
                continue
            if 'pixels' in line:
                continue
            if 'did not converge' in line:
                continue
            if 'on boundry' in line.lower():
                continue
            if 'imfit does not work' in line:
                continue
            if len(line)==1:
                continue
            original_stdout = sys.stdout
            sys.stdout = outfile
            print(line.strip())
            sys.stdout = original_stdout


In [ ]:
#sort the data summary
import numpy as np
import sys
import os

all_lines=[]
textin=f'fitsumfiles/noerrors/fitsummary_{pipetype}_noerrors.txt'

if not os.path.exists('fitsumfiles'):
    os.makedirs('fitsumfiles/sorted')
textout=f'fitsumfiles/sorted/fitsummary_{pipetype}_sorted.txt'

write=True
#write=False

totex=0
with open(textout, 'w') as file:
    if write==True:
        original_stdout = sys.stdout
        sys.stdout = file
    all_lines=[]
    with open(textin, 'r') as infile:
        tline=[]
        for line in infile:
            line = line.split(' ')
            line2=[]
            for i in line:
                if i!='':
                    line2.append(i)
            all_lines.append(line2)
    source_order=[]
    doneindex=[]
    n = len(all_lines)
    for i in range(n):
        if i==0:
            continue
        breaker=False
        for idoneindex in doneindex:
            if i==idoneindex:
                breaker=True
        if breaker==True:
            continue


        isource_order=[]
        isource_order.append(all_lines[i])
        for j in range(n):
            if j==1:
                continue
            if j!=i:
                if all_lines[i][0]==all_lines[j][0]:
                    #so that while iterating through i the source is not repeated
                    isource_order.append(all_lines[j])
                    doneindex.append(j)
            n2 = len(isource_order)


        isource_order = sorted(isource_order, key=lambda x: (
            float(x[2]),  # Primary sorting key remains unchanged.
            # Handle 'NA' in x[3] by using a tuple where 'NA' sorts as a small number (e.g., -1).
            (-1 if x[4] == 'NA' else float(x[4].split('/')[0]) +
            float(x[4].split('/')[1]) / 12 +
            float(x[4].split('/')[2]) / 365),
            # Handle 'NA' in x[4] similarly, making it sort first if present.
            (-1 if x[5] == 'NA' else float(x[5]))
        ))
    

        for i2source_order in isource_order:
            print(i2source_order)
        print('\n')
    if write==True:
        sys.stdout = original_stdout

In [ ]:
#Get Alphas!

source='NGC1068'
freq1flux=54.00
freq2flux=46.31

snr1=6.638
snr2=6.458

freq1fluxunc=snr1/freq1flux
freq2fluxunc=snr2/freq2flux

freq1=215.55
freq2=235.55

alpha=np.log(freq1flux/freq2flux)/np.log(freq1/freq2)
alphaunc=1/np.abs(np.log(freq1/freq2))*np.sqrt((freq1fluxunc/freq1flux)**2+(freq2fluxunc/freq2flux)**2)

print()
print(source)
print(f'{freq1flux:.1f} ± {freq1fluxunc:.1f}')
print(f'{freq2flux:.1f} ± {freq2fluxunc:.1f}')
print(f'{alpha:.2f} ± {alphaunc:.2f}')

source='NGC3079'
freq1flux=33.95
freq2flux=43.84

snr1=14.390
snr2=15.725

freq1fluxunc=snr1/freq1flux
freq2fluxunc=snr2/freq2flux

freq1=215.55
freq2=235.55

alpha=np.log(freq1flux/freq2flux)/np.log(freq1/freq2)
alphaunc=1/np.abs(np.log(freq1/freq2))*np.sqrt((freq1fluxunc/freq1flux)**2+(freq2fluxunc/freq2flux)**2)

print()
print(source)
print(f'{freq1flux:.1f} ± {freq1fluxunc:.1f}')
print(f'{freq2flux:.1f} ± {freq2fluxunc:.1f}')
print(f'{alpha:.2f} ± {alphaunc:.2f}')

source='NGC4258 '
freq1flux=8.53
freq2flux=12.99

snr1=8.186
snr2=8.463

freq1fluxunc=snr1/freq1flux
freq2fluxunc=snr2/freq2flux

freq1=215.55
freq2=235.55

alpha1=np.log(freq1flux/freq2flux)/np.log(freq1/freq2)
alphaunc1=1/np.abs(np.log(freq1/freq2))*np.sqrt((freq1fluxunc/freq1flux)**2+(freq2fluxunc/freq2flux)**2)

print()
print(source)
print(f'{freq1flux:.1f} ± {freq1fluxunc:.1f}')
print(f'{freq2flux:.1f} ± {freq2fluxunc:.1f}')
print(f'{alpha1:.1f} ± {alphaunc1:.1f}')

source='NGC42582'
freq1flux=6.26
freq2flux=6.92

snr1=4.231
snr2=4.507

freq1fluxunc=snr1/freq1flux
freq2fluxunc=snr2/freq2flux

freq1=215.55
freq2=235.55

alpha2=np.log(freq1flux/freq2flux)/np.log(freq1/freq2)
alphaunc2=1/np.abs(np.log(freq1/freq2))*np.sqrt((freq1fluxunc/freq1flux)**2+(freq2fluxunc/freq2flux)**2)

print()
print(source)
print(f'{freq1flux:.1f} ± {freq1fluxunc:.1f}')
print(f'{freq2flux:.1f} ± {freq2fluxunc:.1f}')
print(f'{alpha2:.1f} ± {alphaunc2:.1f}')
print()
avg=(alpha1+alpha2)/2
avgunc=np.sqrt(alphaunc1**2+alphaunc2**2)
print(f'{avg:.1f} ± {avgunc:.1f}')
